# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [24]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [25]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,morristown,36.2140,-83.2949,24.39,80,75,2.06,US,1716051670
1,1,nova sintra,14.8667,-24.7167,23.41,78,6,9.54,CV,1716051936
2,2,fort bragg,35.1390,-79.0060,28.08,71,75,2.06,US,1716051936
3,3,porto novo,6.4965,2.6036,30.90,78,92,3.84,BJ,1716051936
4,4,leh,34.1667,77.5833,6.22,38,57,1.09,IN,1716051936


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [26]:
# %%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng', 
    'Lat', 
    geo=True, 
    tiles='OSM', 
    size='Humidity', 
    line_color='black', 
    xlabel='Longitude', 
    ylabel='Latitude'
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [27]:
# Narrow down cities that fit criteria and drop any results with null values
vacation_df = city_data_df[city_data_df['Max Temp'] < 30]
vacation_df = vacation_df[vacation_df['Humidity'] < 70]
vacation_df = vacation_df[vacation_df['Country'] != 'US']

# Drop any rows with null values
vacation_df.dropna(inplace=True)

# Display sample data
vacation_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,leh,34.1667,77.5833,6.22,38,57,1.09,IN,1716051936
10,10,adamstown,-25.0660,-130.1015,21.76,69,75,11.97,PN,1716051938
12,12,gazojak,41.1875,61.4036,16.24,48,66,8.27,TM,1716051939
13,13,margaret river,-33.9500,115.0667,16.84,44,46,5.29,AU,1716051939
14,14,chegdomyn,51.1167,133.0167,3.60,50,55,3.67,RU,1716051939


### Step 3: Create a new DataFrame called `hotel_df`.

In [28]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = vacation_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
4,4,leh,34.1667,77.5833,6.22,38,57,1.09,IN,1716051936,
10,10,adamstown,-25.0660,-130.1015,21.76,69,75,11.97,PN,1716051938,
12,12,gazojak,41.1875,61.4036,16.24,48,66,8.27,TM,1716051939,
13,13,margaret river,-33.9500,115.0667,16.84,44,46,5.29,AU,1716051939,
14,14,chegdomyn,51.1167,133.0167,3.60,50,55,3.67,RU,1716051939,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [29]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 10
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
leh - nearest hotel: Kang Lha Chen
adamstown - nearest hotel: No hotel found
gazojak - nearest hotel: Жайхун
margaret river - nearest hotel: Margaret River Hotel
chegdomyn - nearest hotel: Бурея
calingasta - nearest hotel: Nora Hôtel
arno - nearest hotel: Pensionat Grisslehamns
stanley - nearest hotel: Hotel 52
xinzhi - nearest hotel: No hotel found
klyuchi - nearest hotel: No hotel found
happy valley-goose bay - nearest hotel: Hotel North Two
kerben - nearest hotel: Asia
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
ingeniero guillermo n. juarez - nearest hotel: Hotel El Indiano
tsuyama - nearest hotel: アルファーワン津山
bage - nearest hotel: hotel obino
tura - nearest hotel: Hotel Polo Orchid
buka - nearest hotel: No hotel found
kapuskasing - nearest hotel: Travelodge
kitwe - nearest hotel: Milford Lodge
lisakovsk - nearest hotel: Оолит
ribeira grande - nearest hotel: Hospedaria JSF
ifaki - nearest hotel: No hotel found
geraldton - nearest hotel: Ocean Centre Ho

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
4,4,leh,34.1667,77.5833,6.22,38,57,1.09,IN,1716051936,Kang Lha Chen
10,10,adamstown,-25.0660,-130.1015,21.76,69,75,11.97,PN,1716051938,No hotel found
12,12,gazojak,41.1875,61.4036,16.24,48,66,8.27,TM,1716051939,Жайхун
13,13,margaret river,-33.9500,115.0667,16.84,44,46,5.29,AU,1716051939,Margaret River Hotel
14,14,chegdomyn,51.1167,133.0167,3.60,50,55,3.67,RU,1716051939,Бурея
...,...,...,...,...,...,...,...,...,...,...,...
574,574,sakiet sidi youssef,36.2229,8.3555,19.64,64,100,2.48,TN,1716052025,No hotel found
582,582,taltal,-25.4000,-70.4833,18.17,64,0,2.83,CL,1716052026,Hostería Taltal
583,583,kaa-khem,51.6833,94.7333,19.67,30,95,2.44,RU,1716052027,No hotel found
584,584,baynjiwayn,35.6183,45.9541,12.28,46,29,0.96,IQ,1716052027,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [30]:
# %%capture --no-display
with_hotel_df = hotel_df[hotel_df['Hotel Name'] != 'No hotel found']

# Configure the map plot
map_plot2 = with_hotel_df.hvplot.points(
    'Lng', 
    'Lat', 
    geo=True, 
    tiles='OSM', 
    size='Humidity', 
    line_color='black', 
    xlabel='Longitude', 
    ylabel='Latitude',
    hover_cols=['Hotel Name', 'Country']
)

# Display the map
map_plot2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name,Country)